This notebook is to retrieve and plot results across several scenarios.

In [ ]:
######################################## Parameters

### List with names for each case
cases_list = [f'ISA_{x}' for x in range(4, -1, -1)]
cases_nice_names_list = ['Low', 'Moderate', 'High', 'Very high', 'Maximum']

### List of networks to consider
network_path_list = [f'results/case_ISA_{x}/networks/base_s_adm___2030.nc' for x in range(4, -1, -1)]

### carrier to consider
carrier = 'onwind'

### Generation feature to consider
feature = 'p_nom_max'



######################################## Derived parameters

### Create a dic with the case name and the network 

dic = dict(zip(cases_list, network_path_list))

In [ ]:
##### Import packages
import pypsa
import pandas as pd
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import yaml
import os 
import sys
import xarray as xr
import seaborn as sns


##### Import local functions
sys.path.append(os.path.abspath(os.path.join('..')))
import functions as xp


##### Read params.yaml
with open('../params.yaml', 'r') as configfile:
    params = yaml.safe_load(configfile)


##### Ignore warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
df_list = []

for case, network_path in dic.items():

    n = pypsa.Network(f'{params["rootpath"]}{network_path}')

    df_local = (n.generators
                .loc[n.generators['carrier']==carrier, ['bus', feature]]
                .rename(columns={feature: case})
                .set_index('bus')
                .div(1e3) # GW
                )

    df_list.append(df_local)


df = pd.concat(df_list, axis=1)



In [ ]:

### sort according to last column
df = df.sort_values(by=df.columns[-1], ascending=False)


### Rename cases
df = df.rename(columns=dict(zip(cases_list, cases_nice_names_list)))

In [ ]:
########## Make plot
fig, ax = plt.subplots(figsize=(12,6))

### Select palette
if carrier == 'onwind':
    ax.set_title('Onshore wind')
    palette = sns.color_palette("Blues", n_colors=len(df.columns))
if carrier == 'solar':
    ax.set_title('Solar PV')
    palette = sns.dark_palette("orange", n_colors=len(df.columns))[::-1]        


### Make plot    
df.T.plot.area(ax=ax, stacked=True, color=palette)


### Get and revert labels in legend
handles, labels = ax.get_legend_handles_labels()
handles = handles[::-1]
labels = labels[::-1]    
ax.legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left')


### 
ax.set_title(f'{carrier}')
ax.set_xlabel('Environmental sensitivity')
ax.set_ylabel('GW')
ax.grid(True, linestyle='--', linewidth=0.5, color='black', alpha=0.15)

fig.tight_layout()


plt.savefig('test.pdf')
